In [12]:
import os 

In [29]:
%pwd

'c:\\Users\\Admin\\Desktop\\source code\\Ttext summarizattion agen using nlp\\Text-Summarization-using-NLP'

In [28]:
os.chdir("Text-Summarization-using-NLP")

In [22]:
from textSummarizer.utils import *
from textSummarizer.utils.common import read_yaml, create_directories
from textSummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from textSummarizer.entity import DataTransformationConfig


In [31]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_tranformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name 
        )

        return data_tranformation_config


In [32]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer 
from datasets import load_dataset,load_from_disk

from textSummarizer.entity import DataTransformationConfig


In [ ]:
class DataTransformation:
    def __init__(self,config : DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length = 1024, truncation = True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'],max_length=1024, truncation =True)

            return{
                'input_ids': input_encodings['input_ids'],
                'attention_mask': input_encodings['attention_mask'],
                'labels' : target_encodings['input_ids']
            }
            
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))
            

In [38]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-11-24 23:16:48,914: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-24 23:16:48,917: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-24 23:16:48,919: INFO: common: created directory at: artifacts]
[2025-11-24 23:16:48,921: INFO: common: created directory at: artifacts/data_transformation]


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 3092.52 examples/s]
